In [7]:
!pip install requests beautifulsoup4 pdfplumber gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [gdown]


In [8]:
import requests
from bs4 import BeautifulSoup
import gdown
import pdfplumber
import os

In [10]:
# -------------------------------------------------
# CONFIG
# -------------------------------------------------

BOOKS_PAGE = "https://jascap.org/cancer-books-pdf/english-books/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

PDF_OUTPUT = "first_book.pdf"
TEXT_OUTPUT = "first_book.txt"

In [11]:
# -------------------------------------------------
# STEP 1: Scrape Google Drive Link
# -------------------------------------------------

print("🔍 Fetching books page...")

response = requests.get(BOOKS_PAGE, headers=HEADERS)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

drive_link = None

for a in soup.find_all("a", href=True):
    href = a["href"]
    if "drive.google.com" in href:
        drive_link = href
        break

if not drive_link:
    raise Exception("❌ No Google Drive link found on page.")

print("✅ First Drive link found:")
print(drive_link)

🔍 Fetching books page...
✅ First Drive link found:
https://drive.google.com/file/d/1bDIxbZtfWZPLvNqeGWr5KnWgeYM-TINz/edit


In [12]:
# -------------------------------------------------
# STEP 2: Download using gdown
# -------------------------------------------------

print("\n⬇ Downloading book from Google Drive...")

gdown.download(drive_link, PDF_OUTPUT, fuzzy=True)

if not os.path.exists(PDF_OUTPUT):
    raise Exception("❌ Download failed.")

print("✅ Download completed:", PDF_OUTPUT)


⬇ Downloading book from Google Drive...


Downloading...
From: https://drive.google.com/uc?id=1bDIxbZtfWZPLvNqeGWr5KnWgeYM-TINz
To: /Users/saideva/Desktop/Cancer_RAG/Experimentation/first_book.pdf
100%|██████████| 3.37M/3.37M [00:01<00:00, 2.05MB/s]

✅ Download completed: first_book.pdf


In [13]:
# -------------------------------------------------
# STEP 3: Extract Text from PDF
# -------------------------------------------------

print("\n📄 Extracting text from PDF...")

full_text = ""

with pdfplumber.open(PDF_OUTPUT) as pdf:
    for page_number, page in enumerate(pdf.pages):
        page_text = page.extract_text()
        if page_text:
            full_text += page_text + "\n"

if not full_text.strip():
    raise Exception("❌ No text extracted. It may be a scanned PDF.")

print("✅ Text extraction completed.")


📄 Extracting text from PDF...
✅ Text extraction completed.


In [16]:
# -------------------------------------------------
# STEP 4: Save Extracted Text
# -------------------------------------------------

with open(TEXT_OUTPUT, "w", encoding="utf-8") as f:
    f.write(full_text)

print("✅ Text saved to:", TEXT_OUTPUT)

# -------------------------------------------------
# Preview Output
# -------------------------------------------------

print("\n📌 First 1500 characters:\n")
print(full_text[10000:15000])

✅ Text saved to: first_book.txt

📌 First 1500 characters:

er) tumors are:
• They are potentially life-threatening.
• Even if removed, they have tendency to grow back.
• They easily invade and damage nearby tissues and organs.
• If the cancer is not diagnosed and treated in time, the
cancer will grow and the cells will typically spread
(metastasize) to distant parts of the body via the blood
stream. Cancer cells break away from the original
(primary) tumor and move away. They not only invade
neighboring organs but also gain access to the
bloodstream or the lymphatic system. These cells then
travel to distant organs and/or regional lymph nodes,
Cancer Simplified | 3
and start growing elsewhere. This process of spread far
and wide using the blood stream or lymphatic system is
called metastasis.
How is the diagnosis of cancer confirmed?
Under the microscope cells usually show a central nucleus,
surrounding nuclear membrane, outer cytoplasm and then a cell
wall. The cytoplasm also contains

In [ ]:




# —————————————————————————————————————————
# STEP 2: Find the first book link
# —————————————————————————————————————————

# This selector may need tweaking if the site structure changes
# Look for first anchor tag that likely points to a PDF
first_pdf_link = None
for a in soup.find_all("a", href=True):
    href = a["href"]
    # Filter for PDF links
    if href.lower().endswith(".pdf"):
        first_pdf_link = href
        break

if not first_pdf_link:
    raise Exception("No PDF link found on the page!")

print("First PDF URL found:", first_pdf_link)

# —————————————————————————————————————————
# STEP 3: Download the PDF
# —————————————————————————————————————————

pdf_response = requests.get(first_pdf_link)
pdf_response.raise_for_status()

# Save temporarily
pdf_path = "first_book.pdf"
with open(pdf_path, "wb") as f:
    f.write(pdf_response.content)

print("PDF downloaded as:", pdf_path)

# —————————————————————————————————————————
# STEP 4: Extract text from PDF
# —————————————————————————————————————————

text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

print("----- BEGIN EXTRACTED TEXT -----")
print(text[:2000])  # Print first 2000 chars
print("----- END OF EXTRACTED TEXT -----")